In [466]:
# =====================================
# IMPORTY
# =====================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [467]:

# =====================================
# PARAMETRY
# =====================================
DATA_PATH = "zbiór_8.csv"
RANDOM_SEED = 1
CORR_THRESHOLD = 0.8
N_BINS = 4
EPS = 1e-6



In [468]:

# =====================================
# WCZYTANIE + SPLIT
# =====================================
df = pd.read_csv(DATA_PATH)

df_train, df_tmp = train_test_split(df, train_size=0.7, stratify=df["default"], random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_tmp, test_size=0.5, stratify=df_tmp["default"], random_state=RANDOM_SEED)

X_train, y_train = df_train.drop(columns="default"), df_train["default"]
X_val, y_val = df_val.drop(columns="default"), df_val["default"]
X_test, y_test = df_test.drop(columns="default"), df_test["default"]

In [469]:
# -------------------------------------
# WRAPPERY DO ZACHOWANIA NAZW KOLUMN
# -------------------------------------
class ColumnNamePreservingPipeline(Pipeline):
    def transform(self, X):
        X_transformed = super().transform(X)
        if isinstance(X_transformed, np.ndarray):
            if hasattr(self.steps[-1][1], "get_feature_names_out"):
                cols = self.steps[-1][1].get_feature_names_out(X.columns)
            else:
                cols = X.columns
            X_transformed = pd.DataFrame(X_transformed, columns=cols, index=X.index)
        return X_transformed

class ColumnNamePreservingCT(ColumnTransformer):
    def transform(self, X):
        X_transformed = super().transform(X)
        cols = []
        for name, trans, cols_in in self.transformers_:
            if trans == 'drop':
                continue
            if hasattr(trans, 'get_feature_names_out'):
                cols.extend(trans.get_feature_names_out(cols_in))
            else:
                cols.extend(cols_in)
        return pd.DataFrame(X_transformed, columns=cols, index=X.index)


In [470]:

# =====================================
# TRANSFORMATORY
# =====================================

class UnimportantDropper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        zero_percentage = (X[:] == 0).sum() / len(X)

        cols_many_zeros = zero_percentage[zero_percentage > 0.5].index.tolist()

        cols_many_zeros = [col for col in cols_many_zeros 
                        if not col.startswith('wsk') 
                        and col != 'pkdKod' ]
        manual_cols=['szczegolnaFormaPrawna_Symbol', 'RP_przeplywy_operacyjne', 'RP_amortyzacja', 'wsk_pokrycie_zob_kr_gotowkowe_2', 'wsk_zadluzenia_gotowki_2', 'wsk_akt_generowania_got_2', 'wsk_pokrycie_wyd_fin_gotowkowe_2', 'wsk_zysk_CF_operacyjny', 'schemat_wsk_bilans']
        
        cols_to_drop = cols_many_zeros + manual_cols
        self.keep_ = [c for c in X.columns if c not in cols_to_drop]
        
        
        return self

    def transform(self, X):
        return X[self.keep_].copy()
    
    def get_feature_names_out(self, input_features=None):
        return getattr(self, 'keep_', input_features)
    
class CategoricalRuleBasedImputer(BaseEstimator, TransformerMixin):
    def __init__(self, cols_to_impute):
        
        self.cols_to_impute = cols_to_impute
        self.modes_ = {}

    def fit(self, X, y=None):
        X = X.copy()
        
        for col in self.cols_to_impute:
            if X[col].notna().sum() > 0:
                self.modes_[col] = X[col].mode()[0]
            else:
                self.modes_[col] = None  # Na wypadek gdyby cała kolumna była NaN
        return self

    def transform(self, X):
        X = X.copy()
        for col, mode_val in self.modes_.items():
            if mode_val is not None:
                X[col] = X[col].replace(0, np.nan)
                X[col] = X[col].fillna(mode_val)
                
        return X
    def get_feature_names_out(self, input_features=None):
        return getattr(self, 'keep_', input_features)
    
        
        

class NumericalRuleBasedImputer(BaseEstimator, TransformerMixin):
    def __init__(self, cols_to_impute):
        self.cols_to_impute = cols_to_impute

    def fit(self, X, y=None):
        self.medians_ = {}
        missing_rates = {}

        wsk_cols = [c for c in X.columns if c.startswith("wsk")]
        for c in wsk_cols:
            r = (X[c].isna().mean() + (X[c] == 0).mean())
            missing_rates[c] = r

        self.low_ = [c for c, r in missing_rates.items() if r < 0.05]
        self.mid_ = [c for c, r in missing_rates.items() if 0.05 <= r < 0.20]
        self.drop_ = [c for c, r in missing_rates.items() if r >= 0.20]

        for col in (self.low_ + self.mid_ + self.cols_to_impute):
            if col in X.columns:
                self.medians_[col] = X[col].replace(0, np.nan).median()

        self.keep_ = [c for c in X.columns if c not in self.drop_]
        return self

    def transform(self, X):
        X = X.copy()
        X = X.drop(columns=self.drop_, errors="ignore")
        for col in (self.low_ + self.mid_ + self.cols_to_impute):
            if col in X.columns:
                X[col] = X[col].replace(0, np.nan).fillna(self.medians_[col])
        return X
    
    def get_feature_names_out(self, input_features=None):
        return getattr(self, 'keep_', input_features)
    

class LogTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        skew = X.skew().abs()
        self.cols_ = skew[skew > 2].index.tolist()
        self.keep_ = X.columns.tolist()
        return self

    def transform(self, X):
        X = X.copy()
        X[self.cols_] = np.log1p(X[self.cols_])
        return X

    def get_feature_names_out(self, input_features=None):
        return getattr(self, 'keep_', input_features)



class OutlierCapper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.no_cap_cols = []
        self.moderate_iqr_cols = []
        self.percentile_cols = []

        for col in X.columns:
            Q1 = X[col].quantile(0.25)
            Q3 =X[col].quantile(0.75)
            IQR = Q3 - Q1
            
            lower_1_5 = Q1 - 1.5 * IQR
            upper_1_5 = Q3 + 1.5 * IQR
            
            below = (X[col] < lower_1_5).sum()
            above = (X[col] > upper_1_5).sum()
            perc_outliers = (below + above) / len(X[col])
            
            if perc_outliers == 0:
                self.no_cap_cols.append(col)
            elif perc_outliers <= 0.1:
                self.moderate_iqr_cols.append(col)
            else:
                self.percentile_cols.append(col)
        return self

    def transform(self, X):
        for col in self.moderate_iqr_cols:
            Q1 = X[col].quantile(0.25)
            Q3 = X[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR
            X[col] = X[col].clip(lower=lower, upper=upper)

        for col in self.percentile_cols:
            lower = X[col].quantile(0.01)
            upper = X[col].quantile(0.99)
            X[col] = X[col].clip(lower=lower, upper=upper)
        return X

    def get_feature_names_out(self, input_features=None):
        return getattr(self, 'keep_', input_features)



class HighCorrelationDropper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.8):
        self.threshold = threshold

    def fit(self, X, y=None):
        Xc = X.copy()
        Xc = Xc[Xc.columns.sort_values()]  

        while True:
            corr = Xc.corr().abs()
            upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

            # znajdź wszystkie pary z korelacją > threshold
            to_drop_pairs = [
                (col1, col2)
                for col1 in upper.columns
                for col2 in upper.index
                if pd.notnull(upper.loc[col1, col2]) and upper.loc[col1, col2] > self.threshold
            ]

            if not to_drop_pairs:
                break

            # zliczamy częstotliwość występowania kolumn w parach do usunięcia
            freq = {}
            for col1, col2 in to_drop_pairs:
                freq[col1] = freq.get(col1, 0) + 1
                freq[col2] = freq.get(col2, 0) + 1

            # wybieramy kolumnę, która występuje najczęściej
            max_freq = max(freq.values())
            max_cols = [col for col, f in freq.items() if f == max_freq]
            col_to_drop = sorted(max_cols)[-1]  # ostatnia alfabetycznie

            Xc = Xc.drop(columns=[col_to_drop])

        self.keep_ = Xc.columns.tolist()
        return self

    def transform(self, X):
        return X[self.keep_].copy()

    def get_feature_names_out(self, input_features=None):
        return getattr(self, 'keep_', input_features)
    
    
class Binner(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.bin_edges_ = {}
        for col in X.columns:
            if X[col].nunique() >= 4:
                try:
                    _, bins = pd.qcut(X[col], q=N_BINS, retbins=True, duplicates='drop')
                    self.bin_edges_[col] = bins
                except:
                    pass
        self.keep_ = X.columns.tolist()
        return self

    def transform(self, X):
        X = X.copy()
        for col, bins in self.bin_edges_.items():
            X[col] = pd.cut(X[col], bins=bins, include_lowest=True)
        return X

    def get_feature_names_out(self, input_features=None):
        return getattr(self, 'keep_', input_features)

class WOETransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y):
        self.maps_ = {}
        tmp = pd.concat([X, y], axis=1)
        for col in X.columns:
            grp = tmp.groupby(col)[y.name].agg(['sum', 'count'])
            grp['good'] = grp['count'] - grp['sum']
            grp['bad_dist'] = grp['sum'] / grp['sum'].sum()
            grp['good_dist'] = grp['good'] / grp['good'].sum()
            grp['woe'] = np.log((grp['good_dist'] + EPS) / (grp['bad_dist'] + EPS))
            self.maps_[col] = grp['woe'].to_dict()
        self.keep_ = X.columns.tolist()
        return self

    def transform(self, X):
        X = X.copy()
        for col in X.columns:
            X[col] = X[col].map(self.maps_[col])
        return X

    def get_feature_names_out(self, input_features=None):
        return getattr(self, 'keep_', input_features)


In [471]:
# =====================================
# PIPELINE
# =====================================
cat_cols = ['formaWlasnosci_Symbol', 'pkdKod', 'schemat_wsk_rzis'] 
num_cols = [col for col in X_train.columns if col not in cat_cols]

num_pipeline = ColumnNamePreservingPipeline([
    ("drop_num", UnimportantDropper()),
    ("imp_num", NumericalRuleBasedImputer(cols_to_impute=[
            'Aktywa', 'Aktywa_trwale', 'Aktywa_obrotowe', 'Naleznosci_krotkoterminowe',
            'Inwestycje_krotkoterminowe', 'Srodki_pieniezne', 'Kapital_wlasny', 'Kapital_podstawowy',
            'Zysk_netto', 'Zobowiazania_rezerwy', 'Zobowiazania_krotkoterminowe',
            'przychody_sprzedazy', 'zysk_sprzedazy', 'pozostale_przychody_oper', 'koszty_operacyjne_pozostale',
            'zysk_operacyjny', 'koszty_finansowe', 'zysk_brutto', 'podatek_dochodowy'
])),
    ("log", LogTransformer()),
    ("cap", OutlierCapper()),
    ("corr", HighCorrelationDropper()),
    ("bin", Binner()),
    ("woe", WOETransformer())
])

cat_pipeline = ColumnNamePreservingPipeline([
    ("drop_cat", UnimportantDropper()),
    ("imp_cat", CategoricalRuleBasedImputer(cols_to_impute=["formaWlasnosci_Symbol"])),
    ("woe_cat", WOETransformer())
])

preprocess = ColumnNamePreservingCT([
    
    ("num", num_pipeline, num_cols),
    ("cat", cat_pipeline, cat_cols) 
])

pipeline = ColumnNamePreservingPipeline([
    ("preprocess", preprocess)
])

In [472]:
# =====================================
# FIT + TRANSFORM
# =====================================
pipeline.fit(X_train, y_train)

X_train_t = pipeline.transform(X_train)
X_val_t = pipeline.transform(X_val)
X_test_t = pipeline.transform(X_test)


In [473]:
X_train_t = X_train_t[sorted(X_train_t.columns)]
X_train_t

,Aktywa,Aktywa_trwale,Inwestycje_krotkoterminowe,Kapital_podstawowy,Naleznosci_krotkoterminowe,Zobowiazania_krotkoterminowe,Zobowiazania_rezerwy,Zysk_netto,formaWlasnosci_Symbol,koszty_finansowe,...,wsk_struktury_kapitalu,wsk_sytuacji_fin,wsk_udzial_kap_obrotowego_w_fin,wsk_udzial_kap_wlasnego_aktywa_2,wsk_udzial_kapitalu_obrotowego_netto,wsk_udzial_zob_biez_sprzedaz_2,wsk_zast_kapitalu_obcego,wsk_zob_s_aktywa_rzeczowe,wsk_zwrot_aktywa_trwale,zysk_netto
904,-0.187852,-0.082025,-0.114212,-0.009966,-0.345062,-0.135564,-0.165126,-0.309417,0.022720,-0.184865,...,0.584091,-0.340297,-0.029717,0.292275,0.043946,0.248961,0.192725,6.269903,-0.152248,-0.342030
1715,0.552592,0.625617,0.782243,0.387764,0.589038,0.376197,0.439645,0.866599,-0.379485,0.450677,...,0.299850,0.439645,0.146918,0.277674,0.349742,0.053973,0.387764,6.269903,-0.152248,0.835685
1449,0.552592,0.625617,0.782243,0.005249,-0.160846,0.376197,0.439645,0.866599,0.022720,-0.998443,...,0.299850,0.439645,0.042856,0.292275,0.349742,0.551240,0.387764,6.269903,-0.152248,0.835685
894,0.448914,0.625617,-0.448901,0.387764,-0.160846,-0.135564,0.439645,NaN,0.022720,0.450677,...,-0.685277,-0.340297,-0.029717,-0.412064,-0.240172,0.248961,-0.459531,6.269903,0.172557,NaN
1575,0.552592,0.625617,0.782243,-0.009966,0.589038,0.376197,0.439645,0.866599,-0.048527,0.450677,...,0.584091,-0.091235,0.146918,0.292275,-0.240172,NaN,-0.459531,6.269903,-0.097204,0.835685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,-0.187852,-0.082025,-0.114212,-0.009966,-0.160846,-0.154117,-0.165126,0.344621,0.022720,-0.998443,...,-0.685277,-0.340297,-0.029717,-0.412064,0.349742,0.551240,0.000000,6.269903,-0.097204,0.214946
2480,-0.187852,-0.293925,-0.114212,0.005249,0.243555,-0.154117,-0.165126,-0.309417,0.022720,-0.184865,...,0.584091,0.439645,0.146918,0.292275,-0.064078,0.248961,0.192725,6.269903,-0.097204,-0.342030
748,-0.187852,-0.082025,-0.114212,-0.236388,-0.345062,-0.135564,-0.165126,-0.309417,0.022720,0.068171,...,0.584091,-0.340297,0.042856,0.292275,0.043946,NaN,0.000000,6.269903,NaN,-0.342030
2866,0.552592,-0.293925,-0.448901,0.387764,0.243555,0.376197,0.439645,0.344621,-0.379485,0.450677,...,0.584091,0.439645,-0.141937,0.292275,0.349742,NaN,0.000000,6.269903,-0.097204,0.214946


In [474]:
# Lista kolumn w kolejności alfabetycznej
columns_sorted = sorted(X_train_t.columns.to_list())

# Wyświetlenie
print(columns_sorted)

['Aktywa', 'Aktywa_trwale', 'Inwestycje_krotkoterminowe', 'Kapital_podstawowy', 'Naleznosci_krotkoterminowe', 'Zobowiazania_krotkoterminowe', 'Zobowiazania_rezerwy', 'Zysk_netto', 'formaWlasnosci_Symbol', 'koszty_finansowe', 'koszty_operacyjne', 'koszty_operacyjne_pozostale', 'pkdKod', 'podatek_dochodowy', 'pozostale_przychody_oper', 'przychody', 'przychody_sprzedazy', 'schemat_wsk_rzis', 'wsk_akt_generowania_got_1', 'wsk_cykl_konwersji_gotowki', 'wsk_ebitda_3', 'wsk_ebitda_aktywa_3', 'wsk_ebitda_koszty_finansowe_3', 'wsk_ebitda_zobowiazan_3', 'wsk_ebitda_zobowiazan_odsetki_3', 'wsk_fin_majatku_kapitalem', 'wsk_koszty_operacyjne', 'wsk_koszty_przychody', 'wsk_liczba_dni_istnienia', 'wsk_marza_ebitda_3', 'wsk_mnoznik_kap_wl', 'wsk_plynnosc_biez_1', 'wsk_plynnosc_biez_2', 'wsk_pokrycie_aktywow_kap_stalym', 'wsk_pokrycie_wyd_fin_gotowkowe_1', 'wsk_pokrycie_zob_kr_gotowkowe_1', 'wsk_rent_operacyjna', 'wsk_struktura_kap_wlasnego_s_2', 'wsk_struktury_finansowania', 'wsk_struktury_kapitalu', 

In [475]:
list1= ['Aktywa', 'Aktywa_trwale', 'Inwestycje_krotkoterminowe', 'Kapital_podstawowy', 'Naleznosci_krotkoterminowe', 'Zobowiazania_krotkoterminowe', 'Zobowiazania_rezerwy', 'Zysk_netto', 'default', 'formaWlasnosci_Symbol', 'koszty_finansowe', 'koszty_operacyjne', 'koszty_operacyjne_pozostale', 'pkdKod', 'podatek_dochodowy', 'pozostale_przychody_oper', 'przychody', 'przychody_sprzedazy', 'schemat_wsk_rzis', 'wsk_akt_generowania_got_1', 'wsk_cykl_konwersji_gotowki', 'wsk_ebitda_aktywa_3', 'wsk_ebitda_koszty_finansowe_3', 'wsk_ebitda_zobowiazan_3', 'wsk_ebitda_zobowiazan_odsetki_3', 'wsk_fin_majatku_kapitalem', 'wsk_koszty_przychody', 'wsk_liczba_dni_istnienia', 'wsk_marza_ebitda_3', 'wsk_ogolnego_zadluzenia_1', 'wsk_plynnosc_biez_2', 'wsk_pokrycie_aktywow_tr_kapitalem_st', 'wsk_pokrycie_wyd_fin_gotowkowe_1', 'wsk_pokrycie_zob_kr_gotowkowe_1', 'wsk_poziom_kapitalu_obrotowego_netto', 'wsk_rent_operacyjna', 'wsk_stopa_marzy_brutto', 'wsk_stopa_zysku_sprzedaz', 'wsk_struktura_kap_wlasnego_1', 'wsk_struktura_kap_wlasnego_s_2', 'wsk_struktury_finansowania', 'wsk_sytuacji_fin', 'wsk_udzial_kap_obrotowego_w_fin', 'wsk_udzial_kapitalu_obrotowego_netto', 'wsk_udzial_zob_biez_sprzedaz_2', 'wsk_zast_kapitalu_obcego', 'wsk_zast_kapitalu_podstawowego', 'wsk_zob_s_aktywa_rzeczowe', 'wsk_zwrot_aktywa_trwale', 'zysk_netto', 'zysk_operacyjny']
list2=columns_sorted
len( list(set(list1) ^ set(list2))) #powinno być ~0

16

In [476]:
list(set(list1) ^ set(list2))

['wsk_pokrycie_aktywow_tr_kapitalem_st',
 'wsk_stopa_marzy_brutto',
 'wsk_pokrycie_aktywow_kap_stalym',
 'wsk_poziom_kapitalu_obrotowego_netto',
 'wsk_plynnosc_biez_1',
 'wsk_stopa_zysku_sprzedaz',
 'wsk_zast_kapitalu_podstawowego',
 'wsk_struktury_kapitalu',
 'zysk_operacyjny',
 'wsk_ogolnego_zadluzenia_1',
 'wsk_struktura_kap_wlasnego_1',
 'wsk_mnoznik_kap_wl',
 'wsk_koszty_operacyjne',
 'wsk_udzial_kap_wlasnego_aktywa_2',
 'default',
 'wsk_ebitda_3']

In [477]:
intersection_with_list1 = list((set(list1) ^ set(list2)) & set(list1))
intersection_with_list1

['wsk_stopa_zysku_sprzedaz',
 'wsk_pokrycie_aktywow_tr_kapitalem_st',
 'wsk_zast_kapitalu_podstawowego',
 'zysk_operacyjny',
 'wsk_ogolnego_zadluzenia_1',
 'wsk_stopa_marzy_brutto',
 'wsk_poziom_kapitalu_obrotowego_netto',
 'wsk_struktura_kap_wlasnego_1',
 'default']

In [478]:
intersection_with_list2 = list((set(list1) ^ set(list2)) & set(list2))
intersection_with_list2

['wsk_struktury_kapitalu',
 'wsk_udzial_kap_wlasnego_aktywa_2',
 'wsk_pokrycie_aktywow_kap_stalym',
 'wsk_mnoznik_kap_wl',
 'wsk_plynnosc_biez_1',
 'wsk_koszty_operacyjne',
 'wsk_ebitda_3']

In [479]:
#TODO  rozbieżność rozmiarowa względem orginalnego kodu

In [480]:
# =====================================
# DEBUG WRAPPER
# =====================================
class DebugTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, transformer, name=None, show_columns_for=None):
        # UWAGA: przypisuj dokładnie to, co przyszło w argumentach
        self.transformer = transformer
        self.name = name
        self.show_columns_for = show_columns_for  # NIE zmieniamy tutaj, nawet jeśli to None!

    def fit(self, X, y=None):
        name = self.name or self.transformer.__class__.__name__
        self.transformer.fit(X, y)
        print(f"[{name} - fit] columns: {X.shape[1]}")
        return self

    def transform(self, X):
        name = self.name or self.transformer.__class__.__name__
        X_trans = self.transformer.transform(X)
        print(f"[{name} - transform] columns: {X_trans.shape[1]}")

        # Dodatkowe logowanie kolumn tylko dla wybranych transformerów
        if self.show_columns_for and name in self.show_columns_for:
            cols = getattr(X_trans, "columns", None)
            if cols is not None:
                print(f"[{name} - columns]: {list(cols)}")
            else:
                print(f"[{name} - no column names available]")
        return X_trans

    def get_feature_names_out(self, input_features=None):
        if hasattr(self.transformer, "get_feature_names_out"):
            return self.transformer.get_feature_names_out(input_features)
        return input_features

# =====================================
# PIPELINES
# =====================================
cat_cols = ['formaWlasnosci_Symbol', 'pkdKod', 'schemat_wsk_rzis'] 
num_cols = [col for col in X_train.columns if col not in cat_cols]

num_pipeline = ColumnNamePreservingPipeline([
    ("drop_num", DebugTransformer(UnimportantDropper(), "drop_num")),
    ("imp_num", DebugTransformer(NumericalRuleBasedImputer(cols_to_impute=[
            'Aktywa', 'Aktywa_trwale', 'Aktywa_obrotowe', 'Naleznosci_krotkoterminowe',
            'Inwestycje_krotkoterminowe', 'Srodki_pieniezne', 'Kapital_wlasny', 'Kapital_podstawowy',
            'Zysk_netto', 'Zobowiazania_rezerwy', 'Zobowiazania_krotkoterminowe',
            'przychody_sprzedazy', 'zysk_sprzedazy', 'pozostale_przychody_oper', 'koszty_operacyjne_pozostale',
            'zysk_operacyjny', 'koszty_finansowe', 'zysk_brutto', 'podatek_dochodowy'
    ]), "imp_num")),
    ("log", DebugTransformer(LogTransformer(), "log")),
    ("cap", DebugTransformer(OutlierCapper(), "cap")),
    ("corr", DebugTransformer(HighCorrelationDropper(), "corr", show_columns_for=["corr"])),
    ("bin", DebugTransformer(Binner(), "bin")),
    ("woe", DebugTransformer(WOETransformer(), "woe"))
])

cat_pipeline = ColumnNamePreservingPipeline([
    ("drop_cat", DebugTransformer(UnimportantDropper(), "drop_cat")),
    ("imp_cat", DebugTransformer(CategoricalRuleBasedImputer(cols_to_impute=["formaWlasnosci_Symbol"]), "imp_cat")),
    ("woe_cat", DebugTransformer(WOETransformer(), "woe_cat"))
])

preprocess = ColumnNamePreservingCT([
    ("num", num_pipeline, num_cols),
    ("cat", cat_pipeline, cat_cols) 
])

pipeline = ColumnNamePreservingPipeline([
    ("preprocess", preprocess)
])


In [481]:
pipeline.fit(X_train, y_train)

[drop_num - fit] columns: 216
[drop_num - transform] columns: 153
[imp_num - fit] columns: 153
[imp_num - transform] columns: 82
[log - fit] columns: 82
[log - transform] columns: 82
[cap - fit] columns: 82
[cap - transform] columns: 82
[corr - fit] columns: 82
[corr - transform] columns: 46
[corr - columns]: ['Aktywa', 'Aktywa_trwale', 'Inwestycje_krotkoterminowe', 'Kapital_podstawowy', 'Naleznosci_krotkoterminowe', 'Zobowiazania_krotkoterminowe', 'Zobowiazania_rezerwy', 'Zysk_netto', 'koszty_finansowe', 'koszty_operacyjne', 'koszty_operacyjne_pozostale', 'podatek_dochodowy', 'pozostale_przychody_oper', 'przychody', 'przychody_sprzedazy', 'wsk_akt_generowania_got_1', 'wsk_cykl_konwersji_gotowki', 'wsk_ebitda_3', 'wsk_ebitda_aktywa_3', 'wsk_ebitda_koszty_finansowe_3', 'wsk_ebitda_zobowiazan_3', 'wsk_ebitda_zobowiazan_odsetki_3', 'wsk_fin_majatku_kapitalem', 'wsk_koszty_operacyjne', 'wsk_koszty_przychody', 'wsk_liczba_dni_istnienia', 'wsk_marza_ebitda_3', 'wsk_mnoznik_kap_wl', 'wsk_plyn

ColumnNamePreservingPipeline(steps=[('preprocess',
                                     ColumnNamePreservingCT(transformers=[('num',
                                                                           ColumnNamePreservingPipeline(steps=[('drop_num',
                                                                                                                DebugTransformer(name='drop_num',
                                                                                                                                 transformer=UnimportantDropper())),
                                                                                                               ('imp_num',
                                                                                                                DebugTransformer(name='imp_num',
                                                                                                                                 transformer=NumericalRuleBasedImputer(cols_to_impute=['Aktywa',
                                                                                                                                                                                       'Aktywa_trwale',
                                                                                                                                                                                       'Aktywa_obrotowe',...
                                                                           ColumnNamePreservingPipeline(steps=[('drop_cat',
                                                                                                                DebugTransformer(name='drop_cat',
                                                                                                                                 transformer=UnimportantDropper())),
                                                                                                               ('imp_cat',
                                                                                                                DebugTransformer(name='imp_cat',
                                                                                                                                 transformer=CategoricalRuleBasedImputer(cols_to_impute=['formaWlasnosci_Symbol']))),
                                                                                                               ('woe_cat',
                                                                                                                DebugTransformer(name='woe_cat',
                                                                                                                                 transformer=WOETransformer()))]),
                                                                           ['formaWlasnosci_Symbol',
                                                                            'pkdKod',
                                                                            'schemat_wsk_rzis'])]))])